# clean up /scratch/aedavids/GTExData/countsGroupedByGene column names

We are having trouble getting spark to create the final count matrix and estimated scaling factors. As a work around we had spark save grouped counts. The columnn names are wrong. instead of  'sum(GTEX-1117F-0226-SM-5GZZ7)' we want  'GTEX-1117F-0226-SM-5GZZ7'

The GTExTrainNumReadsMatrix.tsv was to big to be groupedByGene in one spark batch job. It was broken into smaller batches. We split based on number of columns. These part files will need to be sorted before they can be pasted back together

```
Aedavids@ucsc.edu
1/24/22
```

In [1]:
import pandas as pd

In [2]:
SPARK_HOME="../../sparkBin/spark-3.1.2-bin-hadoop3.2"
import findspark
findspark.init( SPARK_HOME )

In [3]:
from   pyspark.sql.functions import col

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("cleanup") \
    .config("spark.driver.memory", "15g") \
    .getOrCreate()

22/03/24 15:18:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
def fixSumColNames( df ):
    '''
    fix column names. the should be sample names
    # 'sum(ctrl_1)'' should be 'ctrl_1'
    '''

    # skip the the geneId column It is value like gene_2
    oldNames = df.columns[1:]

    sampleNames = ( ( c, c.replace( 'sum(', '' ).replace( ')', '' ) )  for c in oldNames )
    # print(*sampleColNames)

    retDF = df.select( col( 'geneId' ),
                           *( ( col( c ).alias( a ) for c, a in sampleNames ) ) )


    return( retDF )

In [6]:
def fixSumColNamesAndSort(df):
    retDF = fixSumColNames( df ).orderBy('geneId')
    return retDF

# fix the Training batch files
Use unix paste to combine the batches together. 

## first batch
we need to keep the geneId column

In [7]:
rootDir = "scratch-aedavids/GTExData/data/deseq"
x = "/download/trainNumReadsMatrixBatch*/estimateScalingFactorsCLI.py"
y = "/countsGroupedByGene/part*.csv"
p = rootDir + x + y
trainingPartFiles = !! ls $p | sort
trainingPartFiles

['scratch-aedavids/GTExData/data/deseq/download/trainNumReadsMatrixBatch1/estimateScalingFactorsCLI.py/countsGroupedByGene/part-00000-395d0bc2-2daf-4c17-afc1-c46e5a50f51b-c000.csv',
 'scratch-aedavids/GTExData/data/deseq/download/trainNumReadsMatrixBatch2/estimateScalingFactorsCLI.py/countsGroupedByGene/part-00000-ef350e82-35a2-436e-90b1-b444b91ee019-c000.csv',
 'scratch-aedavids/GTExData/data/deseq/download/trainNumReadsMatrixBatch3/estimateScalingFactorsCLI.py/countsGroupedByGene/part-00000-8a7c823c-c70d-4080-ae85-62354f10ce9c-c000.csv',
 'scratch-aedavids/GTExData/data/deseq/download/trainNumReadsMatrixBatch4/estimateScalingFactorsCLI.py/countsGroupedByGene/part-00000-d6da8081-5fd6-471a-88e3-79fcfb37fd29-c000.csv']

In [8]:
firstTrainingBatchPartfile = trainingPartFiles[0] # leave geneId column
remainingTrainingBatchPartFiles = trainingPartFiles[1:] # remove geneId column after sort and before save

In [9]:
def firstTrainingBatch():
    trainingBatch1DF = spark.read.load( firstTrainingBatchPartfile, 
                                        format="csv", 
                                        header="true" ) 

    cleanTrainBatch1DF = fixSumColNamesAndSort(trainingBatch1DF)
    print("cleanTrainBatch1DF numRows:{} numCols:{}".format(cleanTrainBatch1DF.count(), 
                                                            len(cleanTrainBatch1DF.columns)))
    print(cleanTrainBatch1DF.columns[0:3])
    print(cleanTrainBatch1DF.columns[-3:])

    len(cleanTrainBatch1DF.columns)

    cleanTrainBatch1DF.select( cleanTrainBatch1DF.columns[0:3] ).show(3)

    cleanTrainBatch1DF.select( cleanTrainBatch1DF.columns[0:3] ).tail(3)

    batchStr = str(1)
    outFile =  rootDir + "/" "GTExTrainGroupByGenesCountMatrix." + batchStr + ".csv"
    cleanTrainBatch1DF.coalesce(1).write.csv( outFile, mode='overwrite', header=True)
    print("wrote: {}".format(outFile))
    
firstTrainingBatch()

cleanTrainBatch1DF numRows:74777 numCols:2650
['geneId', 'GTEX-1117F-0226-SM-5GZZ7', 'GTEX-1117F-0526-SM-5EGHJ']
['GTEX-14C5O-2526-SM-73KU2', 'GTEX-14C5O-2626-SM-5RQI5', 'GTEX-14C5O-2826-SM-5RQI6']


+----------+------------------------+------------------------+
|    geneId|GTEX-1117F-0226-SM-5GZZ7|GTEX-1117F-0526-SM-5EGHJ|
+----------+------------------------+------------------------+
|      (A)n|                       9|                       1|
|    (AAA)n|                       0|                       0|
|(AAAAAAC)n|                       0|                       0|
+----------+------------------------+------------------------+
only showing top 3 rows



22/03/24 15:18:26 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/03/24 15:18:27 WARN DAGScheduler: Broadcasting large task binary with size 1061.4 KiB
22/03/24 15:18:40 WARN DAGScheduler: Broadcasting large task binary with size 1072.8 KiB
22/03/24 15:18:48 WARN DAGScheduler: Broadcasting large task binary with size 1245.3 KiB


wrote: scratch-aedavids/GTExData/data/deseq/GTExTrainGroupByGenesCountMatrix.1.csv


## sort and clean up the remaining GTExTraining groupby partfiles
remove the geneId column before saving

In [10]:
def remainingTrainingBatch():
    for i in range(len(remainingTrainingBatchPartFiles)):
        batch = remainingTrainingBatchPartFiles[i]
        print("\n********* {}".format(batch))
        trainingBatchDF = spark.read.load( batch, 
                                        format="csv", 
                                        header="true" )    
        cleanTrainBatchDF = fixSumColNamesAndSort(trainingBatchDF)
        # remove the the geneId column. It is the first column
        saveDF = cleanTrainBatchDF.select( cleanTrainBatchDF.columns[1:] )

        print("saveDF numRows:{} numCols:{}".format(saveDF.count(), 
                                                            len(saveDF.columns)))
        print(saveDF.columns[0:3])
        print(saveDF.columns[-3:])

        batchStr = str(i + 2)
        outFile =  rootDir + "/" "GTExTrainGroupByGenesCountMatrix." + batchStr + ".csv"
        saveDF.coalesce(1).write.csv( outFile, mode='overwrite', header=True)
        print("wrote: {}".format(outFile))
        
remainingTrainingBatch()        


********* scratch-aedavids/GTExData/data/deseq/download/trainNumReadsMatrixBatch2/estimateScalingFactorsCLI.py/countsGroupedByGene/part-00000-ef350e82-35a2-436e-90b1-b444b91ee019-c000.csv
saveDF numRows:74777 numCols:2651
['GTEX-14C5O-2926-SM-5RQI1', 'GTEX-14C5O-3026-SM-5YYB2', 'GTEX-14C5O-3126-SM-664NK']
['GTEX-1H3VY-2226-SM-9WYU7', 'GTEX-1H3VY-2326-SM-9WG81', 'GTEX-1H3VY-2426-SM-9WPPD']


22/03/24 15:19:42 WARN DAGScheduler: Broadcasting large task binary with size 1061.6 KiB
22/03/24 15:19:57 WARN DAGScheduler: Broadcasting large task binary with size 1073.0 KiB
22/03/24 15:20:08 WARN DAGScheduler: Broadcasting large task binary with size 1259.1 KiB


wrote: scratch-aedavids/GTExData/data/deseq/GTExTrainGroupByGenesCountMatrix.2.csv

********* scratch-aedavids/GTExData/data/deseq/download/trainNumReadsMatrixBatch3/estimateScalingFactorsCLI.py/countsGroupedByGene/part-00000-8a7c823c-c70d-4080-ae85-62354f10ce9c-c000.csv
saveDF numRows:74777 numCols:2651
['GTEX-1H3VY-2726-SM-A96TL', 'GTEX-1H4P4-0006-SM-AHZ24', 'GTEX-1H4P4-0011-R10b-SM-CE6SG']
['GTEX-QESD-1626-SM-2S1RB', 'GTEX-QESD-1726-SM-2S1R7', 'GTEX-QESD-2026-SM-447BI']


22/03/24 15:21:55 WARN DAGScheduler: Broadcasting large task binary with size 1058.4 KiB
22/03/24 15:22:03 WARN DAGScheduler: Broadcasting large task binary with size 1069.8 KiB
22/03/24 15:22:12 WARN DAGScheduler: Broadcasting large task binary with size 1255.9 KiB


wrote: scratch-aedavids/GTExData/data/deseq/GTExTrainGroupByGenesCountMatrix.3.csv

********* scratch-aedavids/GTExData/data/deseq/download/trainNumReadsMatrixBatch4/estimateScalingFactorsCLI.py/countsGroupedByGene/part-00000-d6da8081-5fd6-471a-88e3-79fcfb37fd29-c000.csv
saveDF numRows:74777 numCols:2460
['GTEX-QLQ7-0005-SM-2S1QP', 'GTEX-QLQ7-0008-SM-447AW', 'GTEX-QLQ7-0526-SM-2I5G3']
['GTEX-ZZPU-1126-SM-5N9CW', 'GTEX-ZZPU-1326-SM-5GZWS', 'GTEX-ZZPU-2726-SM-5NQ8O']


22/03/24 15:24:12 WARN DAGScheduler: Broadcasting large task binary with size 1172.5 KiB


wrote: scratch-aedavids/GTExData/data/deseq/GTExTrainGroupByGenesCountMatrix.4.csv


# Fix the validation and test group by count matrix

In [33]:
validatePartFile = !! ls $rootDir/download/validate/estimateScalingFactorsCLI.py/countsGroupedByGene/part*.csv
validatePartFile = validatePartFile[0]

testPartFile = !! ls $rootDir/download/test/estimateScalingFactorsCLI.py/countsGroupedByGene/part*.csv
testPartFile = testPartFile[0]

print(validatePartFile)
print(testPartFile)

scratch-aedavids/GTExData/data/deseq/download/validate/estimateScalingFactorsCLI.py/countsGroupedByGene/part-00000-78f365fd-60bf-4b2d-a663-0a9a36eb7bbb-c000.csv
scratch-aedavids/GTExData/data/deseq/download/test/estimateScalingFactorsCLI.py/countsGroupedByGene/part-00000-ca9d3ff0-1746-4fec-bf7f-f0d29ccda5ba-c000.csv


In [34]:
def fixBatch( partFile, outFileName):
    print("\n********* {}".format(partFile))
    trainingBatchDF = spark.read.load( partFile, 
                                    format="csv", 
                                    header="true" )    
    
    saveDF = fixSumColNamesAndSort(trainingBatchDF)
    print("saveDF numRows:{} numCols:{}".format(saveDF.count(), 
                                                        len(saveDF.columns)))
    print(saveDF.columns[0:3])
    print(saveDF.columns[-3:])

    outFile =  rootDir + "/" + outFileName + ".csv"
    saveDF.coalesce(1).write.csv( outFile, mode='overwrite', header=True)
    print("wrote: {}".format(outFile))
    
fixBatch( validatePartFile, "GTExValidateGroupByGenesCountMatrix" )
fixBatch( testPartFile, "GTExTestGroupByGenesCountMatrix" )


********* scratch-aedavids/GTExData/data/deseq/download/validate/estimateScalingFactorsCLI.py/countsGroupedByGene/part-00000-78f365fd-60bf-4b2d-a663-0a9a36eb7bbb-c000.csv
saveDF numRows:74777 numCols:3472
['geneId', 'GTEX-1117F-0426-SM-5EGHI', 'GTEX-1117F-0726-SM-5GIEN']
['GTEX-1B932-0226-SM-7PBYH', 'GTEX-1B932-0326-SM-7IGMV', 'GTEX-1B932-0426-SM-73KX4']


22/03/24 15:45:42 WARN DAGScheduler: Broadcasting large task binary with size 1385.8 KiB
22/03/24 15:46:11 WARN DAGScheduler: Broadcasting large task binary with size 1397.3 KiB
22/03/24 15:46:25 WARN DAGScheduler: Broadcasting large task binary with size 1573.0 KiB


wrote: scratch-aedavids/GTExData/data/deseq/GTExValidateGroupByGenesCountMatrix.csv

********* scratch-aedavids/GTExData/data/deseq/download/test/estimateScalingFactorsCLI.py/countsGroupedByGene/part-00000-ca9d3ff0-1746-4fec-bf7f-f0d29ccda5ba-c000.csv
saveDF numRows:74777 numCols:3472
['geneId', 'GTEX-1117F-0226-SM-5GZZ7', 'GTEX-1117F-0526-SM-5EGHJ']
['GTEX-1B932-0726-SM-731EY', 'GTEX-1B932-0826-SM-73KXG', 'GTEX-1B932-0926-SM-73KUP']


22/03/24 15:47:28 WARN DAGScheduler: Broadcasting large task binary with size 1385.9 KiB
22/03/24 15:47:39 WARN DAGScheduler: Broadcasting large task binary with size 1397.3 KiB
22/03/24 15:47:52 WARN DAGScheduler: Broadcasting large task binary with size 1573.0 KiB


wrote: scratch-aedavids/GTExData/data/deseq/GTExTestGroupByGenesCountMatrix.csv
